In [1]:
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import os
import FinanceDataReader as fdr
import requests
from bs4 import BeautifulSoup
import json
from io import BytesIO
from datetime import datetime, timedelta, date
import time
from urllib import parse

In [2]:
chromedriver_autoinstaller.install()

In [3]:
browser = webdriver.Chrome()
url = 'https://finance.naver.com/sise/sise_market_sum.naver?&page='
browser.get(url)

In [4]:
# 거래량, 외국인비율, 상장주식수(천주), 시가총액(억) PER(배) POE(%) 체크박스에 체크가 되어있음.
checkboxes = browser.find_elements(By.NAME, 'fieldIds') # name="fieldIds"에 해당하는 요소를 찾음
for checkbox in checkboxes:
    if checkbox.is_selected(): # 체크박스가 선택된 상태
        checkbox.click() # 체크박스 클릭 (선택 해제)

In [5]:
check_list = ['거래량', 'ROE', 'PER', 'PBR', '외국인비율']

for checkbox in checkboxes:
    parent = checkbox.find_element(By.XPATH, '..') # 부모 element
    label = parent.find_element(By.TAG_NAME, 'label') # 자식 element (label)
    if label.text in check_list:
        checkbox.click()

<a href="javascript:fieldSubmit()"><img src="https://ssl.pstatic.net/imgstock/images5/btn_apply.gif" alt="적용하기" width="55" height="18"></a>

In [6]:
apply_button = browser.find_element(By.XPATH, '//a[@href="javascript:fieldSubmit()"]')
apply_button.click()

In [7]:
index = 'kospi'

In [8]:
# 코스피 선택
try:
    apply_kospi = browser.find_element(By.XPATH, '//a[@href="?sosok=0"]')
    apply_kospi.click()
    index = 'KOSPI'
except:
    print('이미 코스피가 선택되어있습니다.')
    index = 'KOSPI'

이미 코스피가 선택되어있습니다.


In [ ]:
# 코스닥 선택
try:
    apply_kosdaq = browser.find_element(By.XPATH, '//a[@href="?sosok=1"]')
    apply_kosdaq.click()
    index = 'KOSDAQ'
except:
    print('이미 코스닥이 선택되어있습니다.')
    index = 'KOSDAQ'

In [14]:
stock_list = fdr.StockListing('KRX')

In [15]:
index_stock_list = stock_list[stock_list['Market'] == index]

In [16]:
fdr_index_stock_name_list = list(index_stock_list['Name'].unique())

In [17]:
idx = 0
index_stock_name_symbol = {}
while True:
    idx += 1
    browser.get(url + str(idx))
    
    df = pd.read_html(browser.page_source)[1]
    df.dropna(axis = 'index', how = 'all', inplace = True)
    df.dropna(axis = 'columns', how = 'all', inplace = True)
    
    if len(df) == 0:
        break
    
    stock_name_list = list(df['종목명'].unique())
    for name in stock_name_list:
        if name in fdr_index_stock_name_list:
            symbol = index_stock_list.loc[index_stock_list['Name'] == name, 'Code'].values
            index_stock_name_symbol[name] = symbol[0]

In [18]:
browser.quit()

참고 문서

https://infinite-learning.tistory.com/67

https://velog.io/@choi-jiwoo/%EB%84%A4%EC%9D%B4%EB%B2%84-%EA%B8%88%EC%9C%B5-%EA%B3%BC%EA%B1%B0-%EC%A3%BC%EA%B0%80-%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%81%AC%EB%A1%A4%EB%A7%81

https://kmong.com/gig/284822

https://koapy.readthedocs.io/en/latest/notebooks/getting-historical-stock-price-data.html

https://github.com/FinanceData/FinanceDataReader/blob/b28bfb5f802f7ad5ad8012a59c32553225e40040/krx/data.py

In [19]:
class KrxHistoricalDailyPriceDataDownloader:

    def __init__(self):
        self._headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
        }

        self._stocks = None
        self._stocks_delisted = None

        self._bld = 'dbms/MDC/STAT/standard/MDCSTAT01701'

    def get_stocks(self):
        data = {
            'mktsel': 'ALL',
            'typeNo': '0',
            'searchText': '',
            'bld': 'dbms/comm/finder/finder_stkisu',
        }
        url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
        response = requests.post(url, data, headers=self._headers)
        df = pd.json_normalize(response.json()['block1'])
        df = df.set_index('short_code')
        return df

    def get_stocks_delisted(self):
        data = {
            'mktsel': 'ALL',
            'searchText': '',
            'bld': 'dbms/comm/finder/finder_listdelisu',
        }
        url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
        response = requests.post(url, data, headers=self._headers)
        df = pd.json_normalize(response.json()['block1'])
        df = df.set_index('short_code')
        return df

    @property
    def stocks(self):
        if self._stocks is None:
            self._stocks = self.get_stocks()
        return self._stocks

    @property
    def stocks_delisted(self):
        if self._stocks_delisted is None:
            self._stocks_delisted = self.get_stocks_delisted()
        return self._stocks_delisted

    def get_full_code(self, symbol):
        if symbol in self.stocks.index:
            return self.stocks.loc[symbol]['full_code']
        if symbol in self.stocks_delisted.index:
            return self.stocks_delisted.loc[symbol]['full_code']
        raise ValueError('No full_code found for given symbol %s' % symbol)

    def download(self, symbol, start_date=None, end_date=None):
        start_date = datetime(1990,1,1) if start_date==None else pd.to_datetime(start_date)
        end_date = datetime(2100,1,1) if end_date==None else pd.to_datetime(end_date)
        
        full_code = self.get_full_code(symbol)

        url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
        data = {
            'bld': self._bld,
            'isuCd': full_code,
            'isuCd2': '',
            'strtDd': start_date.strftime("%Y%m%d"),
            'endDd': end_date.strftime("%Y%m%d"),
            'share': '1',
            'money': '1',
            'csvxls_isNo': 'false',
        }
        response = requests.post(url, data, headers=self._headers)
        df = pd.json_normalize(response.json()['output'])

        if df.shape[0] == 0:
            return None

        column_names = {
            'TRD_DD':'Date',
            'ISU_CD':'Code',
            'ISU_NM':'Name',
            'MKT_NM':'Market',
            'SECUGRP_NM':'SecuGroup',
            'TDD_CLSPRC':'Close',
            'FLUC_TP_CD':'UpDown',
            'CMPPRVDD_PRC':'Change',
            'FLUC_RT':'ChangeRate',
            'TDD_OPNPRC':'Open',
            'TDD_HGPRC':'High',
            'TDD_LWPRC':'Low',
            'ACC_TRDVOL':'Volume',
            'ACC_TRDVAL':'Amount',
            'MKTCAP':'MarCap',
            'CMPPREVDD_PRC': 'Change',
            'LIST_SHRS': 'Shares',
        }
        df = df.rename(columns=column_names)

        df['Date'] = pd.to_datetime(df['Date'])
        df['ChangeRate'] = pd.to_numeric(df['ChangeRate'].str.replace(',', ''))

        int_columns = [
            'Close',
            'UpDown',
            'Change',
            'Open',
            'High',
            'Low',
            'Volume',
            'Amount',
            'MarCap',
            'Shares'
        ]

        for col in int_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col].str.replace(',', ''), errors='coerce')
                
        #df.set_index('Date', inplace=True)

        return df

In [20]:
def naverNewsUrl(keyword, date=None):
    base_url = f"https://search.naver.com/search.naver?where=news&sm=tab_pge&"
    
    if date is None:
        queryParams = parse.urlencode({
            parse.quote_plus('query'): keyword,
        }, encoding='utf-8')           
    else:
        queryParams = parse.urlencode({
            parse.quote_plus('query'): keyword,
            parse.quote_plus('ds'): parse.unquote(date),
            parse.quote_plus('de'): parse.unquote(date),
        
        }, encoding='utf-8')
      
    url = base_url + queryParams
    return url

In [21]:
def NaverFinancenNewsCrawler(code, y, m, d, now_y, now_m, now_d, max_count):
    start_date = date(y, m, d)
    end_date = date(now_y, now_m, now_d)
    news_df = pd.DataFrame(columns=['Date', 'Code', 'Title'])
    current_date = end_date
    
    browser = webdriver.Chrome()
    update_url = f"https://search.naver.com/search.naver?where=news&query={code}"
    #base_url = naverNewsUrl(code, current_date.strftime("%Y.%m.%d"))
    browser.get(update_url)
    
    while current_date >= start_date:
        base_url = f"https://search.naver.com/search.naver?where=news&query={code}&sort=0&ds={current_date.strftime('%Y.%m.%d')}&de={current_date.strftime('%Y.%m.%d')}&nso=so%3Ar%2Cp%3Afrom{current_date.strftime('%Y%m%d')}to{current_date.strftime('%Y%m%d')}%2Ca%3A&start="
        browser.refresh()
        time.sleep(5)
    
        response = requests.get(base_url).text
        soup = BeautifulSoup(response, 'html.parser')
        news_items = soup.find_all('div', {'class': 'news_area'})
        news_titles = [item.find('a', {'class': 'news_tit'}).text.strip() for item in news_items]
        # news_items = browser.find_element(By.CLASS_NAME, 'news_area')
        # news_titles = [item.find_element(By.CLASS_NAME, 'news_tit').text.strip() for item in news_items]            
        news_titles = news_titles[:max_count]

        news_list = []
        for title in news_titles:
            news_list.append({'Date': current_date.strftime('%Y-%m-%d'), 'Code': code, 'Title': title})
        
        if news_list:
            news_df = pd.concat([news_df, pd.DataFrame(news_list)], ignore_index = True)
        
        current_date -= timedelta(days = 1)

    browser.quit()
        
    return news_df

In [ ]:
#datetime.now().strftime('%Y-%m-%d')

In [23]:
# 과거 주가 데이터 생성 및 공백기간 주가 데이터 업데이트 구현

start_dt = '2022-11-10'
folder_dir = 'my_data/'
#now_date = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))
now_date = pd.to_datetime('2023-11-10')
now_y = str(now_date.year)
now_m = str(now_date.month)
now_d = str(now_date.day)
if len(now_m) < 2:
    now_m = '0'+now_m            
if len(now_d) < 2:
    now_d = '0'+now_d
now_dt = now_y+'-'+now_m+'-'+now_d

number = 0
for name, code in index_stock_name_symbol.items():
    file_path = folder_dir + f'{code}.csv'
    if os.path.isfile(file_path):
        past_df_fdr = pd.read_csv(f'{file_path}.csv')
        if past_df_fdr.index.max() < now_date:
            y = str(df_fdr.index.max().year)
            m = str(df_fdr.index.max().month)
            d = str(df_fdr.index.max().day)
            if len(m) < 2:
                m = '0'+m            
            if len(d) < 2:
                d = '0'+d
            update_start_dt = y+'-'+m+'-'+d
            #df_fdr = fdr.DataReader(code, start = start_date, end = end_date)
            reader = KrxHistoricalDailyPriceDataDownloader()
            df_fdr = reader.download(code, update_start_dt, now_dt)
            df_fdr['Code'] = code
            past_df_fdr = pd.concat([past_df_fdr, df_fdr], axis = 0)
            past_df_fdr.to_csv(f'{file_path}.csv', index = True)
        else:
            continue
    else:
        #df_fdr = fdr.DataReader(code, start = start_date, end = end_date)
        reader = KrxHistoricalDailyPriceDataDownloader()
        df_fdr = reader.download(code, start_dt, now_dt)
        df_fdr['Code'] = code
        df_fdr.to_csv(f'{file_path}.csv', index = True)
    
    number += 1
    
    if number == 10: # 상위 10개 종목만 가져오기 위함
        break

In [26]:
# 과거 주가 종목의 뉴스 데이터 생성 및 공백기간 데이터 업데이트 구현

st_y = 2022
st_m = 11
st_d = 10
folder_dir = 'my_news_data/'
max_count = 5
#now_date = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))
now_date = pd.to_datetime('2023-11-10')
now_y = str(now_date.year)
now_m = str(now_date.month)
now_d = str(now_date.day)
if len(now_m) < 2:
    now_m = '0'+now_m            
if len(now_d) < 2:
    now_d = '0'+now_d
now_dt = now_y+'-'+now_m+'-'+now_d

number = 0
for name, code in index_stock_name_symbol.items():
    print(name, code)
    file_path = folder_dir + f'{code}_news.csv'
    if os.path.isdir(file_path):
        past_df_news = pd.read_csv(f'{file_path}.csv')
        if past_df_news.DATE.max() < now_date:
            y = str(df_fdr.DATE.max().year)
            m = str(df_fdr.DATE.max().month)
            d = str(df_fdr.DATE.max().day)
            if len(m) < 2:
                m = '0'+m            
            if len(d) < 2:
                d = '0'+d
            df_news = NaverFinancenNewsCrawler(code, int(y), int(m), int(d), int(now_y), int(now_m), int(now_d), max_count)
            past_df_news = pd.concat([past_df_news, df_news], axis = 0)
            past_df_news.to_csv(f'{file_path}.csv', index = False)
        else:
            continue
    else:
        df_news = NaverFinancenNewsCrawler(code, int(st_y), int(st_m), int(st_d), int(now_y), int(now_m), int(now_d), max_count)
        df_news.to_csv(f'{file_path}.csv', index = False)
    
    number += 1
    
    if number == 10: # 상위 10개 종목만 가져오기 위함
        break

삼성전자 005930
LG에너지솔루션 373220
SK하이닉스 000660
삼성바이오로직스 207940
삼성전자우 005935
POSCO홀딩스 005490
현대차 005380
LG화학 051910
NAVER 035420
기아 000270
